In [76]:
import os

In [77]:
%pwd

# C:\Users\sunil kumar\OneDrive - AlgoDox Technology PVT LTD\Desktop\dl_ops_champios\vinipro> how to go this directory
os.chdir(r"C:\Users\sunil kumar\OneDrive - AlgoDox Technology PVT LTD\Desktop\dl_ops_champios\vinipro")
%pwd


'C:\\Users\\sunil kumar\\OneDrive - AlgoDox Technology PVT LTD\\Desktop\\dl_ops_champios\\vinipro'

In [78]:
# Data class it import a python tools called data class or no need to manually packge , ceate class 
# that only store data , It reduces the amout of code your need to write 
# no need to write __init__ method , __repr__ method , __eq__ method

# pathlib is impor tthe path class , which helps to your work with file and folder paths to eas;u amd in aclean way for al operating system.


# @dataclass(frozen=True) what is this is a decorator that tells paython that one time fixed that cant chagne the values , ones the obkec is created . it makes the configration to sage from accidental chagne , to Protected if some issues

In [85]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    local_data_file: Path 
    unzip_dir: Path

In [86]:
from src.cnnClassifier.constants import * 
from src.cnnClassifier.utils.common import read_yaml , create_directories 

In [87]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 

        create_directories([self.config.artifacts_root]) 

    def get_data_ingestion_config(self):
        config = self.config.data_ingestion 

        create_directories([config.root_dir]) 
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir = config.unzip_dir 

        )

        return data_ingestion_config

In [88]:
import os 
import urllib.request as request 
import zipfile 
from src.cnnClassifier import logger 
from src.cnnClassifier.utils.common import get_size 

In [89]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} download! with following info: \n {headers}") 
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok=True) 
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [90]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config() 
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file() 
except Exception as e :
    raise e

✅ YAML Content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifiacts/data_ingestion', 'source_url': 'https://github.com/SunilBajiya/ml_ops_champion/row/main/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2025-08-07 14:25:53,474: INFO: common: YAML file config\config.yaml loaded successfully]
✅ YAML Content: {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}
[2025-08-07 14:25:53,479: INFO: common: YAML file params.yaml loaded successfully]
[2025-08-07 14:25:53,481: INFO: common: created directory at: artifacts]
[2025-08-07 14:25:53,483: INFO: common: created directory at: artifiacts/data_ingestion]


HTTPError: HTTP Error 404: Not Found